# Properties calculator and exporter


In [1]:
import geopandas as gpd
import esda
import momepy
import pandas as pd

from sklearn.neighbors import NearestNeighbors

import numpy as np
import rioxarray as rxr
from shapely.ops import nearest_points
from shapely.geometry import MultiLineString

from geopandas.tools import sjoin_nearest



Import building footprints


In [2]:
footprints_shapefile_path = (
    "datafiles\spatial data\Medellin Building Footprints\Medellin Building Footprints.shp"
)

gdf = gpd.read_file(footprints_shapefile_path, engine="pyogrio")

In [3]:
print(gdf.geometry.type.unique())

['Polygon' 'MultiPolygon']


In [4]:
gdf["index_col"] = gdf.index

gdf = gdf.to_crs(epsg=3116)
# Convert Polygons to their centroid points, then extract the coordinates
coords = gdf.geometry.centroid.apply(lambda geom: (geom.x, geom.y)).tolist()

In [5]:
print(gdf.columns)
print(gdf.head())

Index(['tessellate', 'extrude', 'visibility', 'drawOrder', 'icon',
       'boundary_i', 'geometry_w', 'confidence', 'area_in_me', 'bf_source',
       'system_ind', 'geometry', 'index_col'],
      dtype='object')
   tessellate  extrude  visibility  drawOrder  icon boundary_i  \
0          -1        0          -1        NaN  None        191   
1          -1        0          -1        NaN  None        191   
2          -1        0          -1        NaN  None        191   
3          -1        0          -1        NaN  None        191   
4          -1        0          -1        NaN  None        191   

                                          geometry_w confidence  \
0  {"type":"Polygon","coordinates":[[[-75.6045413...       null   
1  {"type":"Polygon","coordinates":[[[-75.6135193...       null   
2  {"type":"Polygon","coordinates":[[[-75.5669423...       null   
3  {"type":"Polygon","coordinates":[[[-75.5981945...       null   
4  {"type":"Polygon","coordinates":[[[-75.5792195...    

Nearest neighbors fit


In [6]:
nn = NearestNeighbors(n_neighbors=6)
nn.fit(coords)

NearestNeighbors(n_neighbors=6)

Importing roads (https://data.humdata.org/dataset/hotosm_col_roads) and DEM (https://data.nasa.gov/) map


In [7]:
roads_shapefile_path = "datafiles\spatial data\Medellin Roads\Medellin Roads.shp"

roads_gdf = gpd.read_file(roads_shapefile_path, engine="pyogrio")

roads_gdf.crs = "EPSG:4326"
roads_gdf = roads_gdf.to_crs(epsg=3116)

roads_gdf["uID"] = momepy.unique_id(roads_gdf)

In [8]:
def convert_multilinestring_to_linestring(geometry):
    if isinstance(geometry, MultiLineString):
        # Ensuring that we handle MultiLineString as iterable
        longest_line = max(geometry.geoms, key=lambda line: line.length)
        return longest_line
    return geometry


roads_gdf["geometry"] = roads_gdf["geometry"].apply(
    convert_multilinestring_to_linestring
)

In [9]:
print(roads_gdf.geometry.type.unique())

['LineString']


In [10]:
print(roads_gdf.head())

         name name_en      highway  surface    smoothness width lanes oneway  \
0  Carrera 58    None  residential  asphalt          good  None     2   None   
1    Calle 24    None  residential  asphalt  intermediate  None     2   None   
2  Carrera 60    None  residential  asphalt  intermediate  None     2   None   
3        None    None      footway  asphalt     excellent  None  None   None   
4    Calle 46    None     tertiary  asphalt     excellent  None     2    yes   

  bridge layer source name_es      osm_id   osm_type  \
0   None  None   None    None  1212895061  ways_line   
1   None  None   None    None   170300076  ways_line   
2   None  None   None    None   180633102  ways_line   
3   None  None   None    None  1132496431  ways_line   
4   None  None   None    None    32525751  ways_line   

                                            geometry  uID  
0  LINESTRING (833487.648 1180349.011, 833487.568...    0  
1  LINESTRING (833287.608 1180424.741, 833290.907...    1  
2 

In [11]:
DEM_geotiff_path = "datafiles\spatial data\DEM.tif"
rds = rxr.open_rasterio(DEM_geotiff_path)
rds.name = "data"
df = rds.squeeze().to_dataframe().reset_index()
geometry = gpd.points_from_xy(df.x, df.y)
DEM_gdf = gpd.GeoDataFrame(df, crs=rds.rio.crs, geometry=geometry)

In [12]:
DEM_gdf = DEM_gdf.to_crs(epsg=3116)

In [13]:
print(DEM_gdf.columns)

Index(['y', 'x', 'band', 'spatial_ref', 'data', 'geometry'], dtype='object')


Integrate tagged footprints information


In [14]:
# Read the CSV file with specific IDs into a DataFrame
csv_path = "datafiles/building-tags.csv"
ids_df = pd.read_csv(csv_path)
# Keep the first occurrence of each duplicate (default behavior)
ids_df = ids_df.drop_duplicates(subset=['system_ind'], keep='first')

# Reset the index of the DataFrame
ids_df.reset_index(drop=True, inplace=True)

In [15]:
# Merge the GeoDataFrame and DataFrame based on "system_ind" column and retain the "tag" column
merged_gdf = gdf.merge(ids_df[["system_ind", "tag"]],
                       on="system_ind", how="inner")

In [16]:
tagged_indices = gdf[gdf["system_ind"].isin(merged_gdf["system_ind"])].index
tagged_coords = np.array(
    gdf.loc[tagged_indices]
    .geometry.centroid.apply(lambda geom: (geom.x, geom.y))
    .tolist()
)

In [17]:
print(merged_gdf.shape)

(3147, 14)


In [18]:
# snap road segmetns to footprints
merged_gdf["road uID"] = momepy.get_network_id(merged_gdf, roads_gdf, "uID")

Snapping:   0%|          | 0/3147 [00:00<?, ?it/s]

C:\Users\rcdog\AppData\Local\Temp\ipykernel_10752\1840026258.py:2: UserWarning: Some objects were not attached to the network. Set larger `min_size``. 2 affected elements.
  merged_gdf["road uID"] = momepy.get_network_id(merged_gdf, roads_gdf, "uID")


In [19]:
print(merged_gdf)

      tessellate  extrude  visibility  drawOrder  icon boundary_i  \
0             -1        0          -1        NaN  None        191   
1             -1        0          -1        NaN  None        191   
2             -1        0          -1        NaN  None        191   
3             -1        0          -1        NaN  None        191   
4             -1        0          -1        NaN  None        191   
...          ...      ...         ...        ...   ...        ...   
3142          -1        0          -1        NaN  None        191   
3143          -1        0          -1        NaN  None        191   
3144          -1        0          -1        NaN  None        191   
3145          -1        0          -1        NaN  None        191   
3146          -1        0          -1        NaN  None        191   

                                             geometry_w confidence  \
0     {"type":"Polygon","coordinates":[[[-75.5981945...       null   
1     {"type":"Polygon","coordi

Integrate GEM classifications of the Duque paper


Roads and elevation calculations


In [20]:
# Create a unary union of road geometries to find nearest points efficiently
roads_unary = roads_gdf.unary_union


# Function to calculate the minimum distance to the nearest road
def min_distance_to_road(geom):
    # Find the nearest point on the unary union of roads and calculate the distance
    nearest_geom = nearest_points(geom, roads_unary)[1]
    return geom.distance(nearest_geom)


# Apply the function across the geometries in the GeoDataFrame
merged_gdf["distance_to_nearest_road"] = merged_gdf.geometry.apply(min_distance_to_road)

In [21]:
# Calculate orientations for all polygons in gdf
gdf["orientation"] = momepy.Orientation(gdf).series
merged_gdf["orientation"] = momepy.Orientation(merged_gdf).series

  0%|          | 0/381406 [00:00<?, ?it/s]

  0%|          | 0/3147 [00:00<?, ?it/s]

In [22]:
# Get the orientations as a numpy array
orientations = gdf["orientation"].values

# Function to calculate average alignment with the 5 nearest neighbors
def calculate_average_alignment_for_merged(merged_gdf, nn_model, orientations):
    alignments = []
    for idx, row in merged_gdf.iterrows():
        # Get the centroid of the current polygon
        geom = row.geometry.centroid
        coords = np.array([[geom.x, geom.y]])
        
        # Find the 5 nearest neighbors in gdf
        distances, neighbor_indices = nn_model.kneighbors(coords, n_neighbors=6)
        
        # Get the orientation of the current polygon
        current_orientation = row["orientation"]
        
        # Get the orientations of the 5 nearest neighbors (excluding the polygon itself)
        neighbor_orientations = orientations[neighbor_indices[0][1:]]
        
        # Calculate the alignment (absolute difference in orientation)
        alignment = np.abs(neighbor_orientations - current_orientation)
        
        # Calculate the average alignment
        average_alignment = np.mean(alignment)
        
        # Append the result to the list
        alignments.append(average_alignment)
    
    return alignments

# Calculate average alignment for each polygon in merged_gdf
average_alignments = calculate_average_alignment_for_merged(merged_gdf, nn, orientations)

# Add the average alignment to the merged_gdf
merged_gdf["average_alignment"] = average_alignments

In [23]:
# Function to calculate alignment with the first nearest neighbor
def calculate_first_neighbor_alignment(merged_gdf, gdf, nn_model):
    alignments = []
    for idx, row in merged_gdf.iterrows():
        # Get the centroid of the current polygon
        geom = row.geometry.centroid
        coords = np.array([[geom.x, geom.y]])
        
        # Find the nearest neighbors in gdf
        distances, neighbor_indices = nn_model.kneighbors(coords, n_neighbors=2)  # 2 because the first one is the polygon itself
        
        # Get the orientation of the current polygon
        current_orientation = row["orientation"]
        
        # Get the orientation of the first nearest neighbor
        first_neighbor_index = neighbor_indices[0][1]  # The first neighbor is the second index
        first_neighbor_orientation = gdf.iloc[first_neighbor_index]["orientation"]
        
        # Calculate the alignment (absolute difference in orientation)
        alignment = abs(current_orientation - first_neighbor_orientation)
        
        # Append the result to the list
        alignments.append(alignment)
    
    return alignments

# Calculate alignment with the first neighbor for each polygon in merged_gdf
first_neighbor_alignments = calculate_first_neighbor_alignment(merged_gdf, gdf, nn)

# Add the alignment with the first neighbor to merged_gdf
merged_gdf["first_neighbor_alignment"] = first_neighbor_alignments

In [24]:
merged_gdf["street_alignment"] = momepy.StreetAlignment(
    merged_gdf,
    roads_gdf,
    "orientation",
    left_network_id="road uID",
    right_network_id="uID",
).series

In [25]:
merged_gdf['centroid'] = merged_gdf.geometry.centroid

# Reset index on your GeoDataFrames before any operation that could potentially duplicate index entries
merged_gdf.reset_index(drop=True, inplace=True)

# Perform the nearest join
merged_with_elevation = sjoin_nearest(merged_gdf, DEM_gdf, how="left", max_distance=None, distance_col="distance")

# It might also be helpful to reset index after the join if necessary
merged_with_elevation.reset_index(drop=True, inplace=True)

merged_gdf['centroid_elevation'] = merged_with_elevation['data']

Nearest neighbor information


In [26]:
distances, indices = nn.kneighbors(tagged_coords)

In [27]:
print(distances.shape)

(3147, 6)


Neighbor related metrics


In [29]:
# Calculate average distance and number of neighbors within 50 meters
average_distances = np.mean(distances[:, 1:], axis=1)

first_neighbor = distances[:, 1]

radius_neighbors = nn.radius_neighbors(
    tagged_coords, radius=50, return_distance=False)
num_neighbors_in_radius = np.array(
    [len(neighbors) for neighbors in radius_neighbors])

merged_gdf["average_distance"] = average_distances
merged_gdf["first_neighbor"] = first_neighbor
merged_gdf["neighbors_within_50m"] = num_neighbors_in_radius

In [30]:
print(len(tagged_indices))  # Expected: 3147
print(len(average_distances))  # Expected: 3147

3147
3147


Polygon specific metrics


In [31]:
# Calculate equivalent rectangular index using momepy
merged_gdf["eri"] = momepy.EquivalentRectangularIndex(merged_gdf).series

# Calculate fractal dimension using momepy
merged_gdf["fractal"] = momepy.FractalDimension(merged_gdf).series

# Calculate convexity using momepy
merged_gdf["convexity"] = momepy.Convexity(merged_gdf).series

merged_gdf["corners"] = momepy.Corners(merged_gdf).series

merged_gdf['perimeter'] = momepy.Perimeter(merged_gdf).series

ccd = momepy.CentroidCorners(merged_gdf)
merged_gdf['ccd_means'] = ccd.mean
merged_gdf['ccd_stdev'] = ccd.std

merged_gdf['cwa'] = momepy.CompactnessWeightedAxis(merged_gdf).series

merged_gdf['elongation'] = momepy.Elongation(merged_gdf).series

merged_gdf['squ_comp'] = momepy.SquareCompactness(merged_gdf).series

merged_gdf['squareness'] = momepy.Squareness(merged_gdf).series

# Calculate the isoperimetric quotient using esda
merged_gdf["iso_q"] = merged_gdf.geometry.apply(
    esda.shape.isoperimetric_quotient
).astype(float)

# Calculate the number of vertices
merged_gdf["num_vertices"] = merged_gdf.geometry.apply(
    lambda geom: len(geom.exterior.coords)
)

  0%|          | 0/3147 [00:00<?, ?it/s]

  0%|          | 0/3147 [00:00<?, ?it/s]

  0%|          | 0/3147 [00:00<?, ?it/s]

In [32]:
merged_gdf

,tessellate,extrude,visibility,drawOrder,icon,boundary_i,geometry_w,confidence,area_in_me,bf_source,...,corners,perimeter,ccd_means,ccd_stdev,cwa,elongation,squ_comp,squareness,iso_q,num_vertices
0,-1,0,-1,NaN,None,191,"{""type"":""Polygon"",""coordinates"":[[[-75.5981945...",null,71.17559519249279,microsoft,...,4,35.009646,6.415137,0.004436,4.463878,0.571417,0.925548,0.086443,0.726923,5
1,-1,0,-1,NaN,None,191,"{""type"":""Polygon"",""coordinates"":[[[-75.5792195...",null,26.4418917136911,microsoft,...,4,20.656604,3.673580,0.011547,2.103012,0.798931,0.987898,0.367310,0.775893,5
2,-1,0,-1,NaN,None,191,"{""type"":""Polygon"",""coordinates"":[[[-75.6044711...",null,144.65328592459315,microsoft,...,4,48.082977,8.510154,0.018698,4.702045,0.903058,0.997583,0.252371,0.783500,5
3,-1,0,-1,NaN,None,191,"{""type"":""Polygon"",""coordinates"":[[[-75.6087693...",null,47.52452662215555,microsoft,...,4,27.740770,4.940326,0.007762,2.851512,0.780641,0.985090,0.182716,0.773688,5
4,-1,0,-1,NaN,None,191,"{""type"":""Polygon"",""coordinates"":[[[-75.5817445...",null,63.01542282907504,microsoft,...,4,32.177006,5.770620,0.018193,3.501991,0.705202,0.970759,0.382407,0.762432,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3142,-1,0,-1,NaN,None,191,"{""type"":""Polygon"",""coordinates"":[[[-75.5680030...",0.8858,284.2757873535156,google,...,4,71.697809,13.403159,0.001789,10.496177,0.488455,0.881715,0.022839,0.692497,5
3143,-1,0,-1,NaN,None,191,"{""type"":""Polygon"",""coordinates"":[[[-75.5687834...",0.8495,288.3638916015625,google,...,4,68.737314,12.315314,0.001569,7.400445,0.716734,0.972798,0.016975,0.764034,5
3144,-1,0,-1,NaN,None,191,"{""type"":""Polygon"",""coordinates"":[[[-75.5712373...",0.858,543.8536987304688,google,...,12,107.000694,14.253321,2.917569,18.601926,0.825711,0.757165,0.020855,0.594676,13
3145,-1,0,-1,NaN,None,191,"{""type"":""Polygon"",""coordinates"":[[[-75.5675224...",0.7433,209.65049743652344,google,...,6,71.330190,9.966417,3.734116,15.584496,0.861754,0.657138,0.050989,0.516115,7


CSV export


In [33]:
print(merged_gdf.columns)

Index(['tessellate', 'extrude', 'visibility', 'drawOrder', 'icon',
       'boundary_i', 'geometry_w', 'confidence', 'area_in_me', 'bf_source',
       'system_ind', 'geometry', 'index_col', 'tag', 'road uID',
       'distance_to_nearest_road', 'orientation', 'average_alignment',
       'first_neighbor_alignment', 'street_alignment', 'centroid',
       'centroid_elevation', 'average_distance', 'first_neighbor',
       'neighbors_within_50m', 'eri', 'fractal', 'convexity', 'corners',
       'perimeter', 'ccd_means', 'ccd_stdev', 'cwa', 'elongation', 'squ_comp',
       'squareness', 'iso_q', 'num_vertices'],
      dtype='object')


In [34]:
columns_to_export = [
    "system_ind",
    "tag",
    "num_vertices",
    "iso_q",
    "fractal",
    "eri",
    "convexity",
    "area_in_me",
    "corners",
    "average_distance",
    "neighbors_within_50m",
    "first_neighbor",
    "distance_to_nearest_road",
    "centroid_elevation",
    "orientation",
    "street_alignment",
    "ccd_means",
    "ccd_stdev",
    "cwa",
    "elongation",
    "squ_comp",
    "squareness",
    "perimeter",
    "average_alignment",
    "first_neighbor_alignment",
]

# Create a CSV file with the selected columns
merged_gdf[columns_to_export].to_csv("datafiles/property_tag.csv", index=False)

print("CSV file created successfully!")

CSV file created successfully!


In [ ]:
print(merged_gdf)